## Lexicase Selection Numba加速

DEAP中Lexicase Selection的默认实现速度较慢。因此，我们可以尝试使用Numba来加速它。
Numba的原理是将Python代码编译为LLVM中间代码，然后再编译为机器码。这可以显著提高Python代码的运行速度。

In [1]:
import numpy as np
import math
import operator

from deap import base, creator, tools, gp


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    
    # 使用numpy创建一个向量
    x = np.linspace(-10, 10, 100) 
    
    return tuple((func(x) - x**2)**2)


# 创建个体和适应度函数，适应度数组大小与数据量相同
creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * 100)  # 假设我们有100个数据点
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

### 遗传算子
在使用Numba进行加速时，需要注意一些特殊的函数可能不受Numba支持，但所有基本的Python运算符都是支持的。

In [2]:
from numba import njit
import numpy as np


@njit(cache=True)
def selAutomaticEpsilonLexicaseNumba(case_values, fit_weights, k):
    selected_individuals = []
    avg_cases = 0

    for i in range(k):
        candidates = list(range(len(case_values)))
        cases = np.arange(len(case_values[0]))
        np.random.shuffle(cases)

        while len(cases) > 0 and len(candidates) > 1:
            errors_for_this_case = np.array(
                [case_values[x][cases[0]] for x in candidates]
            )
            median_val = np.median(errors_for_this_case)
            median_absolute_deviation = np.median(
                np.array([abs(x - median_val) for x in errors_for_this_case])
            )
            if fit_weights > 0:
                best_val_for_case = np.max(errors_for_this_case)
                min_val_to_survive = best_val_for_case - median_absolute_deviation
                candidates = list(
                    [
                        x
                        for x in candidates
                        if case_values[x][cases[0]] >= min_val_to_survive
                    ]
                )
            else:
                best_val_for_case = np.min(errors_for_this_case)
                max_val_to_survive = best_val_for_case + median_absolute_deviation
                candidates = list(
                    [
                        x
                        for x in candidates
                        if case_values[x][cases[0]] <= max_val_to_survive
                    ]
                )
            cases = np.delete(cases, 0)
        avg_cases = (avg_cases * i + (len(case_values[0]) - len(cases))) / (i + 1)
        selected_individuals.append(np.random.choice(np.array(candidates)))
    return selected_individuals, avg_cases

def selAutomaticEpsilonLexicaseFast(individuals, k):
    fit_weights = individuals[0].fitness.weights[0]
    case_values = np.array([ind.fitness.values for ind in individuals])
    index, avg_cases = selAutomaticEpsilonLexicaseNumba(case_values, fit_weights, k)
    selected_individuals = [individuals[i] for i in index]
    return selected_individuals

In [3]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", selAutomaticEpsilonLexicaseFast)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

C:\Users\zhenl\anaconda3\Lib\site-packages\deap\gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


### 演化流程
演化流程与传统符号回归相同。

In [4]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
population = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof, verbose=True)
print(str(hof[0]))


   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max  	min	nevals	std    	avg 	gen	max	min	nevals	std    
0  	300   	2158.77	0  	90000	0  	300   	4085.69	4.14	0  	7  	2  	300   	1.73985
1  	186   	135184 	1  	1.0201e+08	0  	186   	2.64472e+06	5.29667	1  	13 	2  	186   	2.09968
2  	200   	128884 	2  	1.0201e+08	0  	200   	2.60176e+06	5.19333	2  	14 	2  	200   	2.13603
3  	184   	13103.7	3  	6.25e+06  	0  	184   	112630     	4.91333	3  	20 	2  	184   	2.30199
4  	192   	94209.9	4  	1.0201e+08	0  	192   	2.14263e+06	4.62667	4  	13 	2  	192   	2.0217 
5  	182   	165318 	5  	1.1881e+08	0  	182   	3.00756e+06	4.56333	5  	13 	2  	182   	2.01643
6  	183   	46992.4	6  	9.801e+07 	0  	183   	1.48132e+06	4.28333	6  	15 	2  	183   	2.07277
7  	200   	89687.3	7  	1.0201e+08	0  	200   	2.143e+06  	4.33   	7  	15 	2